In [61]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [62]:
print(mnist.train.images.shape)
print(mnist.train.labels.shape)
num_features = mnist.train.images.shape[1]
num_labels = mnist.train.labels.shape[1]
img_size = int(np.sqrt(num_features))
print(img_size)

(55000, 784)
(55000, 10)
28


In [63]:
sess.close()
sess = tf.InteractiveSession()

In [64]:
# Input and output (Placeholder)
x = tf.placeholder(tf.float32, [None, num_features])
y = tf.placeholder(tf.float32, [None, num_labels])

In [65]:
# Model Variables
def weights(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)
def bias(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [66]:
# Layers
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding = 'SAME')
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [67]:
# First convolutional layer
W_conv1 = weights([5, 5, 1, 32]) # Filter kernel: 5x5, 1 input channel (grayscale), 32 output feature maps
b_conv1 = bias([32]) # 32 outputs

# convert images to tensors
x_image = tf.reshape(x, [-1,28, 28, 1])

# First layer
convolve1 = conv2d(x_image, W_conv1) + b_conv1
# Apply RELU (nonlinear activation function mostly used for classification problems)
h_conv1 = tf.nn.relu(convolve1)
# Apply max pooling
h_pool1 = max_pool_2x2(h_conv1)

layer1 = h_pool1

In [68]:
# Second convolutional layer
W_conv2 = weights([5, 5, 32, 64]) # Filter kernel: 5x5, 32 input channels (first layer), 64 output feature maps
b_conv2 = bias([64]) # 64 outputs

convolve2 = conv2d(layer1, W_conv2) + b_conv2
h_conv2 = tf.nn.relu(convolve2)
h_pool2 = max_pool_2x2(h_conv2)
layer2 = h_pool2

In [69]:
# Third layer = Fully connected layer
W_fc1 = weights([7*7*64, 1024])
b_fc1 = bias([1024])

#Flattening
layer2_matrix = tf.reshape(layer2, [-1, 7*7*64])
# Apply weights
matmul_fc1 = tf.matmul(layer2_matrix, W_fc1) + b_fc1
# Apply RELU
h_fc1 = tf.nn.relu(matmul_fc1)

layer3 = h_fc1

In [70]:
# Optional dropout to reduce overfitting
keep_prob = tf.placeholder(tf.float32)
layer3_drop = tf.nn.dropout(layer3, keep_prob)

In [71]:
# Final layer: Softmax, Fully connected layer
W_fc2 = weights([1024, 10]) # 10 classes for digist 0,1,2,3,4,5,6,7,8,9
b_fc2 = bias([10])
matmul_fc2 = tf.matmul(layer3_drop, W_fc2) + b_fc2
# Apply softmax activation function
y_conv = tf.nn.softmax(matmul_fc2)
layer4 = y_conv

In [72]:
# Define Loss and Optimizer
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(layer4), axis=1))
train = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [73]:
correct_prediction = tf.equal(tf.argmax(layer4, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [74]:
sess.run(tf.global_variables_initializer())
epochs = 1000
for i in range(epochs):
    batch = mnist.train.next_batch(50)
    if i % 100 == 0:
        train_acc = accuracy.eval(feed_dict={x: batch[0], y: batch[1], keep_prob: 1.0})
        print("step {0}, training acc {1}".format(i, train_acc))
    train.run(feed_dict={x: batch[0], y: batch[1], keep_prob: 0.5})

step 0, training acc 0.03999999910593033
step 100, training acc 0.8799999952316284
step 200, training acc 0.9599999785423279
step 300, training acc 0.9200000166893005
step 400, training acc 0.9599999785423279
step 500, training acc 1.0
step 600, training acc 0.9399999976158142
step 700, training acc 0.9800000190734863
step 800, training acc 0.9599999785423279
step 900, training acc 1.0


In [75]:
# Evaluat model, evaluate in batches to avoid out of memory issues
n_batches = mnist.test.images.shape[0] // 50
cumulative_acc = 0.0
for i in range(n_batches):
    batch = mnist.test.next_batch(50)
    cumulative_acc += accuracy.eval(feed_dict={x: batch[0], y: batch[1], keep_prob: 1.0})
print("test accuracy: {0}".format(cumulative_acc / n_batches))

test accuracy: 0.9637000003457069


In [76]:
sess.close()